# Human Analysis

## Preparation

In [2]:
# Imports
import pandas as pd
import numpy as np
import pathlib
import sys
import os
import re
from typing import Dict, List

# Get the current working directory
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from Import import get_file_paths_pathlib
from Models import ModelType
from Helpers import clean_prediction
from Database_Query import query_blazegraph

In [11]:
predictions_folder_path = pathlib.PurePath("../Results/csv/Predictions/Test/")
dataset_folder_path = pathlib.PurePath("../SPICE_dataset_pp/test/")
general_evaluation_path = pathlib.PurePath("../Results/csv/Evaluations/Test/summary_general_evaluation_v02.csv")
output_path = pathlib.PurePath("./spice_merged_data.xlsx")

general_evaluation_df = pd.read_csv(general_evaluation_path)
predictions_files = get_file_paths_pathlib(predictions_folder_path)

samples_per_category = 15

In [ ]:
# Get all question types
question_types = general_evaluation_df["question_type"].unique()
print(f'Question types: {question_types}')
print('-' * 50)
print(f'Number of samples per model-prompt combination: {len(question_types) * samples_per_category}')

In [13]:
# Get all model-prompt combinations
model_promtps = []
exclude_models = [ModelType.LLAMALORA, ModelType.BASELINE, ModelType.DAVINCI]
for model in ModelType:
    if model in exclude_models:
        continue
    for prompt in ["zero-shot", "few-shot"]:
        model_promtps.append((model, prompt))

print(model_promtps)

[(<ModelType.LLAMA: 'llama-7b'>, 'zero-shot'), (<ModelType.LLAMA: 'llama-7b'>, 'few-shot'), (<ModelType.LORA: 'lora-7b'>, 'zero-shot'), (<ModelType.LORA: 'lora-7b'>, 'few-shot'), (<ModelType.VICUNA: 'vicuna-7b'>, 'zero-shot'), (<ModelType.VICUNA: 'vicuna-7b'>, 'few-shot'), (<ModelType.GPT3: 'gpt-3.5-turbo-0613'>, 'zero-shot'), (<ModelType.GPT3: 'gpt-3.5-turbo-0613'>, 'few-shot')]


### Create initial file for one model-prompt combination

In [14]:
def _get_first_values_of_dict(entities: str) -> Dict[str, str]:
    # Get the first 3 values of the dictionary
    result = {}
    dictionary = eval(entities)

    for i, (key, value) in enumerate(dictionary.items()):
        if i == 3:
            break
        result[key] = value
    return result


def create_conversation_history(turnID: str) -> str:
    # turnID format: "test#QA_72#QA_42#4"
    folder = turnID.split("#")[1]
    file = turnID.split("#")[2] + ".csv"
    
    # Select the corresponding file from the dataset_folder
    conv_history_file = pathlib.PurePath(dataset_folder_path, folder, file)
    conv_history_df = pd.read_csv(conv_history_file)
    
    # Iterate line by line over the dataframe
    conv_history = []
    for _, row in conv_history_df.iterrows():
        if row["turnID"] == turnID:
            break

        speaker = row['speaker']
        
        if speaker == "USER":
            conv_history.append(f"{speaker}: {row['utterance']} \t\t entities: {row['entities_in_utterance']}, \t relations: {row['relations']}, \t types: {row['type_list']}")
        else:
            response_entities = _get_first_values_of_dict(row['entities_in_utterance'])
            if len(response_entities) > 0:
                conv_history.append(f"{speaker}: {row['utterance']} \t\t entities: {response_entities}")
            else:
                conv_history.append(f"{speaker}: {row['utterance']} \t\t entities: -")

    # Remove all \n from the strings
    conv_history = [s.replace("\n", "") for s in conv_history]

    # Merge conv_history into single string separated by newlines
    return "\n".join(conv_history)

create_conversation_history("test#QA_91#QA_68#12")

"USER: What is the gene that encodes rRNA 2'-O-methyltransferase fibrillarin ? \t\t entities: {'Q5446439': 'Fibrillarin'}, \t relations: {'P702': 'encoded by'}, \t types: {'Q863908': 'nucleic acid sequence'}\nSYSTEM: FBL \t\t entities: {'Q14914390': 'FBL'}\nUSER: Which biological process is rRNA 2'-O-methyltransferase fibrillarin involved in ? \t\t entities: {'Q5446439': 'Fibrillarin'}, \t relations: {'P682': 'biological process'}, \t types: {'Q2996394': 'biological process'}\nSYSTEM: methylation \t\t entities: {'Q518328': 'methylation'}\nUSER: And how about Methyltransferase-like protein 25? \t\t entities: {'Q21119436': 'Methyltransferase like 25'}, \t relations: {}, \t types: {'Q2996394': 'biological process'}\nSYSTEM: methylation \t\t entities: {'Q518328': 'methylation'}\nUSER: Where is that nutrient found ? \t\t entities: {'Q21119436': 'Methyltransferase like 25'}, \t relations: {'P703': 'found in taxon'}, \t types: {'Q16521': 'taxon'}\nSYSTEM: Homo sapiens \t\t entities: {'Q159786

In [15]:
merged_data_df = pd.DataFrame(columns=["turnID", "question_type", "description", "conversation_history", "question", "gold_query"])
model = model_promtps[0][0].value
prompt = model_promtps[0][1]
filtered_predictions_files = [file_path for file_path in predictions_files if model in file_path.parts[-1] and prompt in file_path.parts[-1]]

for question_type in question_types:
    missing_samples = samples_per_category
    while missing_samples > 0:
        # Select random file from prediction_files
        random_file = np.random.choice(filtered_predictions_files)
        prediction_df = pd.read_csv(random_file)

        # Get the turnIDs of all rows that have the current question type
        turnIDs = prediction_df[prediction_df["question_type"] == question_type]["turnID"].unique()

        # Remove values that are already in merged_data_df
        existing_turnIDs = merged_data_df["turnID"].values
        turnIDs = [turnID for turnID in turnIDs if turnID not in existing_turnIDs]

        if len(turnIDs) == 0:
            continue

        # Select a random turnID
        turnID = np.random.choice(turnIDs)

        # Get the two rows with the selected turnID
        rows = prediction_df[prediction_df["turnID"] == turnID]

        # Extract relevant values
        turnID = rows.iloc[0]["turnID"]
        question_type = rows.iloc[0]["question_type"]
        description = rows.iloc[0]["description"]
        conversation_history = create_conversation_history(turnID)
        text = f"{rows.iloc[0]['utterance']} \t\t entities: {rows.iloc[0]['entities_in_utterance']}, \t relations: {rows.iloc[0]['relations']}, \t types: {rows.iloc[0]['type_list']}"
        gold_query = rows.iloc[1]["sparql_query"]
        prediction = clean_prediction(rows.iloc[0][f"prediction_{model}"])

        # There does not exist a SPARQL query for clarifications -> we use the utterance instead as this would be the expected model output
        if question_type == "Clarification":
            gold_query = rows.iloc[1]["utterance"]

        new_row = {"turnID": turnID, "question_type": question_type, "description": description, "conversation_history": conversation_history, "question": text, "gold_query": gold_query, f"prediction_{model}_{prompt}": prediction}

        # Add a single new row to merged_data_df with the extracted values
        merged_data_df = pd.concat([merged_data_df, pd.DataFrame([new_row])], ignore_index=True)
        missing_samples -= 1

merged_data_df.head()

,turnID,question_type,description,conversation_history,question,gold_query,prediction_llama-7b_zero-shot
0,test#QA_72#QA_41#0,Simple Question (Direct),Simple Question,,What is Boyd Okwuonu a member of ? \t\t entiti...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,What is Boyd Okwuonu a member of ? What is Boy...
1,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...
2,test#QA_258#QA_74#0,Simple Question (Direct),Simple Question,,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,What is the metabolic process that hypothetica...
3,test#QA_100#QA_84#9,Simple Question (Direct),Simple Question|Single Entity,USER: Who has the surname Thomas Schneider ? \...,Which administrative territories are sister to...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,Input question: Which administrative territori...
4,test#QA_38#QA_30#5,Simple Question (Direct),Simple Question|Single Entity,USER: Where did Božidar Borko expire ? \t\t en...,Who is author of Dungeons & Dragons Master Rul...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,Input question: Who is author of Dungeons & Dr...


### Add all other model-prompt combinations to the file

In [16]:
# Iterate over all model-prompt combinations, skipping the first one that was already processed
for model, prompt in model_promtps[1:]:
    # Iterate over all rows in merged_data_df
    for index, row in merged_data_df.iterrows():
        try:
            # Get the turnID of the current row
            turnID = row["turnID"]
            folder = turnID.split("#")[1]
            file = f'{turnID.split("#")[2]}_{model.value}_{prompt}-chat-history.csv'

            # Select the corresponding file from the predictions_folder
            prediction_file = pathlib.PurePath(predictions_folder_path, folder, file)
            prediction_df = pd.read_csv(prediction_file)

            # Get the two rows with the selected turnID
            rows = prediction_df[prediction_df["turnID"] == turnID]

            # Extract relevant values
            prediction = clean_prediction(rows.iloc[0][f"prediction_{model.value}"])

            # Add the prediction to the current row
            merged_data_df.at[index, f"prediction_{model.value}_{prompt}"] = prediction
        except:
            print(f'Model: {model.value}, Prompt: {prompt}, TurnID: {turnID}')
            continue

merged_data_df.head()

,turnID,question_type,description,conversation_history,question,gold_query,prediction_llama-7b_zero-shot,prediction_llama-7b_few-shot,prediction_lora-7b_zero-shot,prediction_lora-7b_few-shot,prediction_vicuna-7b_zero-shot,prediction_vicuna-7b_few-shot,prediction_gpt-3.5-turbo-0613_zero-shot,prediction_gpt-3.5-turbo-0613_few-shot
0,test#QA_72#QA_41#0,Simple Question (Direct),Simple Question,,What is Boyd Okwuonu a member of ? \t\t entiti...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,What is Boyd Okwuonu a member of ? What is Boy...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,SELECT ?x WHERE { ?x wdt:P54/wdt:memberOf/wdt:...,SELECT ?x WHERE { ?x wdt:P54 wd:Q19901566 . },SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...
1,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,SELECT ?occupation WHERE { ?x wdt:P106 ?occupa...,ASK { wd:Q641091 wdt:P106 wd:Q12737077 . },SELECT ?occupationLabel WHERE { wd:Q641091 wdt...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...
2,test#QA_258#QA_74#0,Simple Question (Direct),Simple Question,,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,SELECT ?process WHERE { ?x wdt:P682 ?process ....,ASK { wd:Q23572309 wdt:P682 wd:Q1057 . },SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...
3,test#QA_100#QA_84#9,Simple Question (Direct),Simple Question|Single Entity,USER: Who has the surname Thomas Schneider ? \...,Which administrative territories are sister to...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,Input question: Which administrative territori...,SELECT ?x WHERE { ?x wdt:P190 wd:Q572545 . },SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,SELECT ?territory WHERE { ?territory wdt:P190 ...,SELECT ?x WHERE { ?x wdt:P190 wd:Q572545 . },SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...
4,test#QA_38#QA_30#5,Simple Question (Direct),Simple Question|Single Entity,USER: Where did Božidar Borko expire ? \t\t en...,Who is author of Dungeons & Dragons Master Rul...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,Input question: Who is author of Dungeons & Dr...,SELECT ?x WHERE { wd:Q5315182 wdt:P800 ?x . ?x...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,SELECT ?x WHERE { wd:Q5315182 wdt:P800 ?x . ?x...,SELECT ?author WHERE { ?x 'Q5315182'@en . ?x n...,ASK { wd:Q5315182 wdt:P800 . },SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...


In [17]:
# Export the dataframe to an Excel file
merged_data_df.to_excel(output_path, index=False)

### Add Syntax check

In [18]:
# load the file
syntax_df = pd.read_excel(output_path)

syntax_df.head()

,turnID,question_type,description,conversation_history,question,gold_query,prediction_llama-7b_zero-shot,prediction_llama-7b_few-shot,prediction_lora-7b_zero-shot,prediction_lora-7b_few-shot,prediction_vicuna-7b_zero-shot,prediction_vicuna-7b_few-shot,prediction_gpt-3.5-turbo-0613_zero-shot,prediction_gpt-3.5-turbo-0613_few-shot
0,test#QA_72#QA_41#0,Simple Question (Direct),Simple Question,NaN,What is Boyd Okwuonu a member of ? \t\t entiti...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,What is Boyd Okwuonu a member of ? What is Boy...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,SELECT ?x WHERE { ?x wdt:P54/wdt:memberOf/wdt:...,SELECT ?x WHERE { ?x wdt:P54 wd:Q19901566 . },SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...
1,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,SELECT ?occupation WHERE { ?x wdt:P106 ?occupa...,ASK { wd:Q641091 wdt:P106 wd:Q12737077 . },SELECT ?occupationLabel WHERE { wd:Q641091 wdt...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...
2,test#QA_258#QA_74#0,Simple Question (Direct),Simple Question,NaN,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,SELECT ?process WHERE { ?x wdt:P682 ?process ....,ASK { wd:Q23572309 wdt:P682 wd:Q1057 . },SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...
3,test#QA_100#QA_84#9,Simple Question (Direct),Simple Question|Single Entity,USER: Who has the surname Thomas Schneider ? \...,Which administrative territories are sister to...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,Input question: Which administrative territori...,SELECT ?x WHERE { ?x wdt:P190 wd:Q572545 . },SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,SELECT ?territory WHERE { ?territory wdt:P190 ...,SELECT ?x WHERE { ?x wdt:P190 wd:Q572545 . },SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...
4,test#QA_38#QA_30#5,Simple Question (Direct),Simple Question|Single Entity,USER: Where did Božidar Borko expire ? \t\t en...,Who is author of Dungeons & Dragons Master Rul...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,Input question: Who is author of Dungeons & Dr...,SELECT ?x WHERE { wd:Q5315182 wdt:P800 ?x . ?x...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,SELECT ?x WHERE { wd:Q5315182 wdt:P800 ?x . ?x...,SELECT ?author WHERE { ?x 'Q5315182'@en . ?x n...,ASK { wd:Q5315182 wdt:P800 . },SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...


In [19]:
def add_db_query_results(df: pd.DataFrame, model: ModelType, prompt: str) -> pd.DataFrame:
    query_execution_successes: List[str] = []
    query_results: List[str] = []
    query_errors: List[str] = []

    for row in df[f'prediction_{model.value}_{prompt}']:
            query_blazegraph_result = query_blazegraph(row)
            query_execution_success = query_blazegraph_result[0]
            query_execution_successes.append(query_execution_success)

            if query_execution_success:
                query_results.append(query_blazegraph_result[1])
                query_errors.append("")
            else:
                query_errors.append(query_blazegraph_result[1])
                query_results.append("")

    insert_index = df.columns.get_loc(f'prediction_{model.value}_{prompt}')
    df.insert(loc=insert_index + 1, column=f'{model.value}_{prompt}_query_success', value=query_execution_successes)
    df.insert(loc=insert_index + 2, column=f'{model.value}_{prompt}_query_results', value=query_results)
    df.insert(loc=insert_index + 3, column=f'{model.value}_{prompt}_query_errors', value=query_errors)
    return df


# syntax_df = add_db_query_results(syntax_df, model_promtps[6][0], model_promtps[6][1])

In [20]:
# Add the query results to the dataframe for each model-prompt combination
for model, prompt in model_promtps:
    syntax_df = add_db_query_results(syntax_df, model, prompt)

syntax_df.head()

,turnID,question_type,description,conversation_history,question,gold_query,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,llama-7b_zero-shot_query_errors,...,vicuna-7b_few-shot_query_results,vicuna-7b_few-shot_query_errors,prediction_gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_zero-shot_query_success,gpt-3.5-turbo-0613_zero-shot_query_results,gpt-3.5-turbo-0613_zero-shot_query_errors,prediction_gpt-3.5-turbo-0613_few-shot,gpt-3.5-turbo-0613_few-shot_query_success,gpt-3.5-turbo-0613_few-shot_query_results,gpt-3.5-turbo-0613_few-shot_query_errors
0,test#QA_72#QA_41#0,Simple Question (Direct),Simple Question,NaN,What is Boyd Okwuonu a member of ? \t\t entiti...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,What is Boyd Okwuonu a member of ? What is Boy...,False,,"Error: HTTPConnectionPool(host='localhost', po...",...,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,,"Error: HTTPConnectionPool(host='localhost', po..."
1,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,False,,"Error: HTTPConnectionPool(host='localhost', po...",...,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?occupationLabel WHERE { wd:Q641091 wdt...,False,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,False,,"Error: HTTPConnectionPool(host='localhost', po..."
2,test#QA_258#QA_74#0,Simple Question (Direct),Simple Question,NaN,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,What is the metabolic process that hypothetica...,False,,"Error: HTTPConnectionPool(host='localhost', po...",...,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,,"Error: HTTPConnectionPool(host='localhost', po..."
3,test#QA_100#QA_84#9,Simple Question (Direct),Simple Question|Single Entity,USER: Who has the surname Thomas Schneider ? \...,Which administrative territories are sister to...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,Input question: Which administrative territori...,False,,"Error: HTTPConnectionPool(host='localhost', po...",...,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,,"Error: HTTPConnectionPool(host='localhost', po..."
4,test#QA_38#QA_30#5,Simple Question (Direct),Simple Question|Single Entity,USER: Where did Božidar Borko expire ? \t\t en...,Who is author of Dungeons & Dragons Master Rul...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,Input question: Who is author of Dungeons & Dr...,False,,"Error: HTTPConnectionPool(host='localhost', po...",...,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,,"Error: HTTPConnectionPool(host='localhost', po..."


In [21]:
# export the dataframe to an Excel file
output_path_v02 = pathlib.PurePath("./spice_merged_data_v02.xlsx")
syntax_df.to_excel(output_path_v02, index=False)

### Add Entity Reference Check

In [22]:
 # load the file
entity_check_df = pd.read_excel(output_path_v02)

entity_check_df.head()

,turnID,question_type,description,conversation_history,question,gold_query,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,llama-7b_zero-shot_query_errors,...,vicuna-7b_few-shot_query_results,vicuna-7b_few-shot_query_errors,prediction_gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_zero-shot_query_success,gpt-3.5-turbo-0613_zero-shot_query_results,gpt-3.5-turbo-0613_zero-shot_query_errors,prediction_gpt-3.5-turbo-0613_few-shot,gpt-3.5-turbo-0613_few-shot_query_success,gpt-3.5-turbo-0613_few-shot_query_results,gpt-3.5-turbo-0613_few-shot_query_errors
0,test#QA_72#QA_41#0,Simple Question (Direct),Simple Question,NaN,What is Boyd Okwuonu a member of ? \t\t entiti...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,What is Boyd Okwuonu a member of ? What is Boy...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po..."
1,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?occupationLabel WHERE { wd:Q641091 wdt...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po..."
2,test#QA_258#QA_74#0,Simple Question (Direct),Simple Question,NaN,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,What is the metabolic process that hypothetica...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po..."
3,test#QA_100#QA_84#9,Simple Question (Direct),Simple Question|Single Entity,USER: Who has the surname Thomas Schneider ? \...,Which administrative territories are sister to...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,Input question: Which administrative territori...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po..."
4,test#QA_38#QA_30#5,Simple Question (Direct),Simple Question|Single Entity,USER: Where did Božidar Borko expire ? \t\t en...,Who is author of Dungeons & Dragons Master Rul...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,Input question: Who is author of Dungeons & Dr...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po..."


In [23]:
def extract_entities_from_text(text: str) -> List[str]:
    # Extract all substrings from the text using regex that start with Q followed by an arbitrary number of digits
    return re.findall(r'Q\d+', f"{text}")

entities = extract_entities_from_text("""
                           USER: What is the school where James DePreist was educated ? 		 entities: {'Q1356135': 'James DePreist'}, 	 relations: {'P69': 'educated at'}, 	 types: {'Q3914': 'school'}
SYSTEM: The Wharton School 		 entities: {'Q1329269': 'The Wharton School'}
USER: What was that person a member of ? 		 entities: {'Q1356135': 'James DePreist'}, 	 relations: {'P463': 'member of'}, 	 types: {'Q811430': 'constructions'}
SYSTEM: American Academy of Arts and Sciences 		 entities: {'Q463303': 'American Academy of Arts and Sciences'}
USER: What caused the death of that person ? 		 entities: {'Q1356135': 'James DePreist'}, 	 relations: {'P509': 'cause of death'}, 	 types: {'Q2057971': 'health problem'}
SYSTEM: myocardial infarction 		 entities: {'Q12152': 'myocardial infarction'}
USER: How many health problems do around the same number of people suffer from as that health problem ? 		 entities: {}, 	 relations: {'P1050': 'medical condition'}, 	 types: {'Q2057971': 'health problem', 'Q502895': 'common name'}
SYSTEM: 217 		 entities: -

                           """)
print(entities)

['Q1356135', 'Q3914', 'Q1329269', 'Q1356135', 'Q811430', 'Q463303', 'Q1356135', 'Q2057971', 'Q12152', 'Q2057971', 'Q502895']


In [24]:
def add_reference_check(df: pd.DataFrame, model: ModelType, prompt: str) -> pd.DataFrame:
    unexpected_references: List[str] = []

    for _, row in df.iterrows():
        unreferenced_entities: List[str] = []
        entities = []
        # Extract all entities from the question
        entities.extend(extract_entities_from_text(row["question"]))
        # Extract all entities from the conversation history
        entities.extend(extract_entities_from_text(row["conversation_history"]))
        # Extract all entities from the gold query
        entities.extend(extract_entities_from_text(row["gold_query"]))
        # Extract all entities from the prediction
        current_prediction_entities = extract_entities_from_text(row[f'prediction_{model.value}_{prompt}'])

        # Remove duplicates
        entities = list(set(entities))
        current_prediction_entities = list(set(current_prediction_entities))

        # Check if all entities from the prediction are in the list of entities
        unreferenced_entities.append([entity for entity in current_prediction_entities if entity not in entities])
        # Flat the list
        unreferenced_entities = [item for sublist in unreferenced_entities for item in sublist]

        # Add it to the new column
        unexpected_references.append(unreferenced_entities)

    # Add the unreferenced entities to the dataframe
    insert_index = df.columns.get_loc(f'prediction_{model.value}_{prompt}')
    df.insert(loc=insert_index + 4, column=f'{model.value}_{prompt}_unexpected_references', value=unexpected_references)
    return df

# add_reference_check(entity_check_df, model_promtps[6][0], model_promtps[6][1])
# entity_check_df.head()

In [25]:
# Add the reference check to the dataframe for each model-prompt combination
for model, prompt in model_promtps:
    entity_check_df = add_reference_check(entity_check_df, model, prompt)

entity_check_df.head()

,turnID,question_type,description,conversation_history,question,gold_query,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,llama-7b_zero-shot_query_errors,...,prediction_gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_zero-shot_query_success,gpt-3.5-turbo-0613_zero-shot_query_results,gpt-3.5-turbo-0613_zero-shot_query_errors,gpt-3.5-turbo-0613_zero-shot_unexpected_references,prediction_gpt-3.5-turbo-0613_few-shot,gpt-3.5-turbo-0613_few-shot_query_success,gpt-3.5-turbo-0613_few-shot_query_results,gpt-3.5-turbo-0613_few-shot_query_errors,gpt-3.5-turbo-0613_few-shot_unexpected_references
0,test#QA_72#QA_41#0,Simple Question (Direct),Simple Question,NaN,What is Boyd Okwuonu a member of ? \t\t entiti...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,What is Boyd Okwuonu a member of ? What is Boy...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
1,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?occupationLabel WHERE { wd:Q641091 wdt...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
2,test#QA_258#QA_74#0,Simple Question (Direct),Simple Question,NaN,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,What is the metabolic process that hypothetica...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
3,test#QA_100#QA_84#9,Simple Question (Direct),Simple Question|Single Entity,USER: Who has the surname Thomas Schneider ? \...,Which administrative territories are sister to...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,Input question: Which administrative territori...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
4,test#QA_38#QA_30#5,Simple Question (Direct),Simple Question|Single Entity,USER: Where did Božidar Borko expire ? \t\t en...,Who is author of Dungeons & Dragons Master Rul...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,Input question: Who is author of Dungeons & Dr...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[Q5]


In [26]:
# export the dataframe to an Excel file
output_path_v03 = pathlib.PurePath("./spice_merged_data_v03.xlsx")
entity_check_df.to_excel(output_path_v03, index=False)

### Add Gold Result

In [27]:
# load the file
final_df = pd.read_excel(output_path_v03)
final_df.head()

,turnID,question_type,description,conversation_history,question,gold_query,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,llama-7b_zero-shot_query_errors,...,prediction_gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_zero-shot_query_success,gpt-3.5-turbo-0613_zero-shot_query_results,gpt-3.5-turbo-0613_zero-shot_query_errors,gpt-3.5-turbo-0613_zero-shot_unexpected_references,prediction_gpt-3.5-turbo-0613_few-shot,gpt-3.5-turbo-0613_few-shot_query_success,gpt-3.5-turbo-0613_few-shot_query_results,gpt-3.5-turbo-0613_few-shot_query_errors,gpt-3.5-turbo-0613_few-shot_unexpected_references
0,test#QA_72#QA_41#0,Simple Question (Direct),Simple Question,NaN,What is Boyd Okwuonu a member of ? \t\t entiti...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,What is Boyd Okwuonu a member of ? What is Boy...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
1,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?occupationLabel WHERE { wd:Q641091 wdt...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
2,test#QA_258#QA_74#0,Simple Question (Direct),Simple Question,NaN,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,What is the metabolic process that hypothetica...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
3,test#QA_100#QA_84#9,Simple Question (Direct),Simple Question|Single Entity,USER: Who has the surname Thomas Schneider ? \...,Which administrative territories are sister to...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,Input question: Which administrative territori...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
4,test#QA_38#QA_30#5,Simple Question (Direct),Simple Question|Single Entity,USER: Where did Božidar Borko expire ? \t\t en...,Who is author of Dungeons & Dragons Master Rul...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,Input question: Who is author of Dungeons & Dr...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",...,SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",['Q5']


In [28]:
def add_db_gold_query_result(df: pd.DataFrame) -> pd.DataFrame:
    query_results: List[str] = []

    for row in df[f'gold_query']:
            query_blazegraph_result = query_blazegraph(row)
            query_execution_success = query_blazegraph_result[0]

            if query_execution_success:
                query_results.append(query_blazegraph_result[1])
            else:
                query_results.append("")

    insert_index = df.columns.get_loc(f'gold_query')
    df.insert(loc=insert_index + 1, column=f'gold_result', value=query_results)
    return df

add_db_gold_query_result(final_df)
final_df.head()

,turnID,question_type,description,conversation_history,question,gold_query,gold_result,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,...,prediction_gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_zero-shot_query_success,gpt-3.5-turbo-0613_zero-shot_query_results,gpt-3.5-turbo-0613_zero-shot_query_errors,gpt-3.5-turbo-0613_zero-shot_unexpected_references,prediction_gpt-3.5-turbo-0613_few-shot,gpt-3.5-turbo-0613_few-shot_query_success,gpt-3.5-turbo-0613_few-shot_query_results,gpt-3.5-turbo-0613_few-shot_query_errors,gpt-3.5-turbo-0613_few-shot_unexpected_references
0,test#QA_72#QA_41#0,Simple Question (Direct),Simple Question,NaN,What is Boyd Okwuonu a member of ? \t\t entiti...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,,What is Boyd Okwuonu a member of ? What is Boy...,False,NaN,...,SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q19901566 wdt:P54 ?x . ?x...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
1,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,,Input question: What does Damian Chmiel do for...,False,NaN,...,SELECT ?occupationLabel WHERE { wd:Q641091 wdt...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
2,test#QA_258#QA_74#0,Simple Question (Direct),Simple Question,NaN,What is the metabolic process that hypothetica...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,,What is the metabolic process that hypothetica...,False,NaN,...,SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q23572309 wdt:P682 ?x . ?...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
3,test#QA_100#QA_84#9,Simple Question (Direct),Simple Question|Single Entity,USER: Who has the surname Thomas Schneider ? \...,Which administrative territories are sister to...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,,Input question: Which administrative territori...,False,NaN,...,SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?x WHERE { wd:Q572545 wdt:P190 ?x . ?x ...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[]
4,test#QA_38#QA_30#5,Simple Question (Direct),Simple Question|Single Entity,USER: Where did Božidar Borko expire ? \t\t en...,Who is author of Dungeons & Dragons Master Rul...,SELECT ?x WHERE { ?x wdt:P800 wd:Q5315182 . ?x...,,Input question: Who is author of Dungeons & Dr...,False,NaN,...,SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",[],SELECT ?author WHERE { wd:Q5315182 wdt:P800 ?w...,False,NaN,"Error: HTTPConnectionPool(host='localhost', po...",['Q5']


In [29]:
# export the dataframe to an Excel file
output_path_v04 = pathlib.PurePath("./spice_merged_data_v04.xlsx")
final_df.to_excel(output_path_v04, index=False)

## Aggregate Results

In [3]:
# load the file
sample_size = 150
file_name = "spice_label_data_v02.xlsx"
labeled_path = pathlib.PurePath("./", file_name)
labeled_df = pd.read_excel(labeled_path)
labeled_df = labeled_df.fillna('')
labeled_df.head()

,turnID,question_type,description,conversation_history,question,gold_result,gold_query,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,...,gpt-3.5-turbo-0613_few-shot_unexpected_references,llama-7b_zero-shot,llama-7b_few-shot,lora-7b_zero-shot,lora-7b_few-shot,vicuna-7b_zero-shot,vicuna-7b_few-shot,gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_few-shot,comment
0,test#QA_162#QA_14#2,Simple Question (Direct),Simple Question|Single Entity,USER: Which is the cell component of 16S rRNA ...,Which gene encodes 16S rRNA methyltransferase ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q23487488 wdt:P702 ?x . ?...,Input question: What is the nucleic acid seque...,False,,...,[],o,-,-,-,-,-,i,d,
1,test#QA_54#QA_63#0,Simple Question (Direct),Simple Question,,Which nonprofit organization is Peter Zumthor ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q123179 wdt:P69 ?x . ?x w...,Generate a SPARQL query that answers the given...,False,,...,[],o,-,-,-,le,il,-,-,Vicuna-zero-shot: Uses entity reference starti...
2,test#QA_55#QA_90#11,Simple Question (Direct),Simple Question|Single Entity,USER: What was awarded to Oskar Saier ? \t\t e...,Which person had Jenna Wolfe as their girlfrie...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q6177791 wdt:P451 ?x . ?x...,Input question: Which person had Jenna Wolfe a...,False,,...,['Q5'],o,-,d,d,i,-,d,i,GPT-few-shot only uses first digit of one refe...
3,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,False,,...,[],o,-,-,-,ie,i,l,-,Vicuna-zero-shot: Uses entity reference starti...
4,test#QA_140#QA_65#6,Simple Question (Direct),Simple Question|Single Entity,USER: Which person was the father of Friedrich...,Who has Bremen as his/her tomb ? \t\t entities...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { ?x wdt:P119 wd:Q24879 . ?x w...,Input question: Which political territory is t...,False,,...,[],o,i,-,-,il,i,-,-,LLaMA-few-shot: swappt triple ordering (i.e. ?...


In [4]:
model_prompt_combinations = ["llama-7b_zero-shot", "llama-7b_few-shot", "lora-7b_zero-shot", "lora-7b_few-shot", "vicuna-7b_zero-shot", "vicuna-7b_few-shot", "gpt-3.5-turbo-0613_zero-shot", "gpt-3.5-turbo-0613_few-shot"]

In [5]:
# Check all instances where syntax is incorrect and add corresponding letter
for index, row in labeled_df.iterrows():
    for model_prompt_combination in model_prompt_combinations:
        if not row[f"{model_prompt_combination}_query_success"]:
            labeled_df.at[index, f"{model_prompt_combination}"] = (row[model_prompt_combination] + "s") if "s" not in row[model_prompt_combination] else row[model_prompt_combination]

labeled_df.head()

,turnID,question_type,description,conversation_history,question,gold_result,gold_query,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,...,gpt-3.5-turbo-0613_few-shot_unexpected_references,llama-7b_zero-shot,llama-7b_few-shot,lora-7b_zero-shot,lora-7b_few-shot,vicuna-7b_zero-shot,vicuna-7b_few-shot,gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_few-shot,comment
0,test#QA_162#QA_14#2,Simple Question (Direct),Simple Question|Single Entity,USER: Which is the cell component of 16S rRNA ...,Which gene encodes 16S rRNA methyltransferase ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q23487488 wdt:P702 ?x . ?...,Input question: What is the nucleic acid seque...,False,,...,[],os,-,-,-,-s,-,i,d,
1,test#QA_54#QA_63#0,Simple Question (Direct),Simple Question,,Which nonprofit organization is Peter Zumthor ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q123179 wdt:P69 ?x . ?x w...,Generate a SPARQL query that answers the given...,False,,...,[],os,-,-,-,les,il,-,-,Vicuna-zero-shot: Uses entity reference starti...
2,test#QA_55#QA_90#11,Simple Question (Direct),Simple Question|Single Entity,USER: What was awarded to Oskar Saier ? \t\t e...,Which person had Jenna Wolfe as their girlfrie...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q6177791 wdt:P451 ?x . ?x...,Input question: Which person had Jenna Wolfe a...,False,,...,['Q5'],os,-,d,d,i,-,d,i,GPT-few-shot only uses first digit of one refe...
3,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,False,,...,[],os,-,-,-,ie,i,ls,-,Vicuna-zero-shot: Uses entity reference starti...
4,test#QA_140#QA_65#6,Simple Question (Direct),Simple Question|Single Entity,USER: Which person was the father of Friedrich...,Who has Bremen as his/her tomb ? \t\t entities...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { ?x wdt:P119 wd:Q24879 . ?x w...,Input question: Which political territory is t...,False,,...,[],os,i,-,-,il,i,-,-,LLaMA-few-shot: swappt triple ordering (i.e. ?...


In [6]:
# Check all instances where unreferenced entites are used and add letter
for index, row in labeled_df.iterrows():
    for model_prompt_combination in model_prompt_combinations:
        if row[f"{model_prompt_combination}_unexpected_references"] != "[]":
            labeled_df.at[index, f"{model_prompt_combination}"] = (row[model_prompt_combination] + "e") if "e" not in row[model_prompt_combination] else row[model_prompt_combination]

labeled_df.head()

,turnID,question_type,description,conversation_history,question,gold_result,gold_query,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,...,gpt-3.5-turbo-0613_few-shot_unexpected_references,llama-7b_zero-shot,llama-7b_few-shot,lora-7b_zero-shot,lora-7b_few-shot,vicuna-7b_zero-shot,vicuna-7b_few-shot,gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_few-shot,comment
0,test#QA_162#QA_14#2,Simple Question (Direct),Simple Question|Single Entity,USER: Which is the cell component of 16S rRNA ...,Which gene encodes 16S rRNA methyltransferase ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q23487488 wdt:P702 ?x . ?...,Input question: What is the nucleic acid seque...,False,,...,[],os,-,-,-,-s,-,i,d,
1,test#QA_54#QA_63#0,Simple Question (Direct),Simple Question,,Which nonprofit organization is Peter Zumthor ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q123179 wdt:P69 ?x . ?x w...,Generate a SPARQL query that answers the given...,False,,...,[],os,-,-,-,les,il,-,-,Vicuna-zero-shot: Uses entity reference starti...
2,test#QA_55#QA_90#11,Simple Question (Direct),Simple Question|Single Entity,USER: What was awarded to Oskar Saier ? \t\t e...,Which person had Jenna Wolfe as their girlfrie...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q6177791 wdt:P451 ?x . ?x...,Input question: Which person had Jenna Wolfe a...,False,,...,['Q5'],os,-,d,d,i,-,d,ie,GPT-few-shot only uses first digit of one refe...
3,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,False,,...,[],os,-,-,-,ie,i,ls,-,Vicuna-zero-shot: Uses entity reference starti...
4,test#QA_140#QA_65#6,Simple Question (Direct),Simple Question|Single Entity,USER: Which person was the father of Friedrich...,Who has Bremen as his/her tomb ? \t\t entities...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { ?x wdt:P119 wd:Q24879 . ?x w...,Input question: Which political territory is t...,False,,...,[],os,i,-,-,il,i,-,-,LLaMA-few-shot: swappt triple ordering (i.e. ?...


In [7]:
# o should be mutually exclusive
for index, row in labeled_df.iterrows():
    for model_prompt_combination in model_prompt_combinations:
        if 'o' in row[model_prompt_combination]:
            labeled_df.at[index, model_prompt_combination] = "o"

labeled_df.head()

,turnID,question_type,description,conversation_history,question,gold_result,gold_query,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,...,gpt-3.5-turbo-0613_few-shot_unexpected_references,llama-7b_zero-shot,llama-7b_few-shot,lora-7b_zero-shot,lora-7b_few-shot,vicuna-7b_zero-shot,vicuna-7b_few-shot,gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_few-shot,comment
0,test#QA_162#QA_14#2,Simple Question (Direct),Simple Question|Single Entity,USER: Which is the cell component of 16S rRNA ...,Which gene encodes 16S rRNA methyltransferase ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q23487488 wdt:P702 ?x . ?...,Input question: What is the nucleic acid seque...,False,,...,[],o,-,-,-,-s,-,i,d,
1,test#QA_54#QA_63#0,Simple Question (Direct),Simple Question,,Which nonprofit organization is Peter Zumthor ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q123179 wdt:P69 ?x . ?x w...,Generate a SPARQL query that answers the given...,False,,...,[],o,-,-,-,les,il,-,-,Vicuna-zero-shot: Uses entity reference starti...
2,test#QA_55#QA_90#11,Simple Question (Direct),Simple Question|Single Entity,USER: What was awarded to Oskar Saier ? \t\t e...,Which person had Jenna Wolfe as their girlfrie...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q6177791 wdt:P451 ?x . ?x...,Input question: Which person had Jenna Wolfe a...,False,,...,['Q5'],o,-,d,d,i,-,d,ie,GPT-few-shot only uses first digit of one refe...
3,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,False,,...,[],o,-,-,-,ie,i,ls,-,Vicuna-zero-shot: Uses entity reference starti...
4,test#QA_140#QA_65#6,Simple Question (Direct),Simple Question|Single Entity,USER: Which person was the father of Friedrich...,Who has Bremen as his/her tomb ? \t\t entities...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { ?x wdt:P119 wd:Q24879 . ?x w...,Input question: Which political territory is t...,False,,...,[],o,i,-,-,il,i,-,-,LLaMA-few-shot: swappt triple ordering (i.e. ?...


In [8]:
#replace "-" in all cells with "" for each column with name model_prompt_combination
for model_prompt_combination in model_prompt_combinations:
    labeled_df[model_prompt_combination] = labeled_df[model_prompt_combination].str.replace("-", "")
    labeled_df[model_prompt_combination] = labeled_df[model_prompt_combination].str.replace(" ", "")

labeled_df.head()

,turnID,question_type,description,conversation_history,question,gold_result,gold_query,prediction_llama-7b_zero-shot,llama-7b_zero-shot_query_success,llama-7b_zero-shot_query_results,...,gpt-3.5-turbo-0613_few-shot_unexpected_references,llama-7b_zero-shot,llama-7b_few-shot,lora-7b_zero-shot,lora-7b_few-shot,vicuna-7b_zero-shot,vicuna-7b_few-shot,gpt-3.5-turbo-0613_zero-shot,gpt-3.5-turbo-0613_few-shot,comment
0,test#QA_162#QA_14#2,Simple Question (Direct),Simple Question|Single Entity,USER: Which is the cell component of 16S rRNA ...,Which gene encodes 16S rRNA methyltransferase ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q23487488 wdt:P702 ?x . ?...,Input question: What is the nucleic acid seque...,False,,...,[],o,,,,s,,i,d,
1,test#QA_54#QA_63#0,Simple Question (Direct),Simple Question,,Which nonprofit organization is Peter Zumthor ...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q123179 wdt:P69 ?x . ?x w...,Generate a SPARQL query that answers the given...,False,,...,[],o,,,,les,il,,,Vicuna-zero-shot: Uses entity reference starti...
2,test#QA_55#QA_90#11,Simple Question (Direct),Simple Question|Single Entity,USER: What was awarded to Oskar Saier ? \t\t e...,Which person had Jenna Wolfe as their girlfrie...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q6177791 wdt:P451 ?x . ?x...,Input question: Which person had Jenna Wolfe a...,False,,...,['Q5'],o,,d,d,i,,d,ie,GPT-few-shot only uses first digit of one refe...
3,test#QA_272#QA_45#8,Simple Question (Direct),Simple Question|Single Entity,USER: Which administrative territory is the bi...,What does Damian Chmiel do for a living ? \t\t...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { wd:Q641091 wdt:P106 ?x . ?x ...,Input question: What does Damian Chmiel do for...,False,,...,[],o,,,,ie,i,ls,,Vicuna-zero-shot: Uses entity reference starti...
4,test#QA_140#QA_65#6,Simple Question (Direct),Simple Question|Single Entity,USER: Which person was the father of Friedrich...,Who has Bremen as his/her tomb ? \t\t entities...,"{""sparql"": {""@xmlns"": ""http://www.w3.org/2005/...",SELECT ?x WHERE { ?x wdt:P119 wd:Q24879 . ?x w...,Input question: Which political territory is t...,False,,...,[],o,i,,,il,i,,,LLaMA-few-shot: swappt triple ordering (i.e. ?...


In [9]:
# iterate through each model and print metrics
print(file_name)
for model in model_prompt_combinations:
    letter_counts = {}
    letter_combinations = {}
    dataset_issues = 0

    for text in labeled_df[model][:sample_size]:
        if "!" in text:
            dataset_issues += 1
            continue

        if text in letter_combinations:
            letter_combinations[text] += 1
        else:
            letter_combinations[text] = 1

        for letter in text:
            if letter in letter_counts:
                letter_counts[letter] += 1
            else:
                letter_counts[letter] = 1
    print("issue type distribution of", model)
    print("samples ignored due to dataset issues:", dataset_issues)

    # exclude dataset issues from evaluation
    valid_sample_size = sample_size - dataset_issues

    for key in letter_counts:
        percentage = letter_counts[key]/valid_sample_size
       
        print(key, ":", letter_counts[key], "\t all :", percentage)
    
    print("Letter combinations:")
    for letter_combination in letter_combinations:
        print(letter_combination, ":", letter_combinations[letter_combination], "\t all :", letter_combinations[letter_combination]/valid_sample_size)
    print("..."*10)

spice_label_data_v02.xlsx
issue type distribution of llama-7b_zero-shot
samples ignored due to dataset issues: 6
o : 144 	 all : 1.0
Letter combinations:
o : 144 	 all : 1.0
..............................
issue type distribution of llama-7b_few-shot
samples ignored due to dataset issues: 6
i : 89 	 all : 0.6180555555555556
o : 15 	 all : 0.10416666666666667
s : 20 	 all : 0.1388888888888889
e : 7 	 all : 0.04861111111111111
Letter combinations:
 : 20 	 all : 0.1388888888888889
i : 89 	 all : 0.6180555555555556
o : 15 	 all : 0.10416666666666667
se : 7 	 all : 0.04861111111111111
s : 13 	 all : 0.09027777777777778
..............................
issue type distribution of lora-7b_zero-shot
samples ignored due to dataset issues: 6
d : 1 	 all : 0.006944444444444444
i : 10 	 all : 0.06944444444444445
e : 2 	 all : 0.013888888888888888
o : 15 	 all : 0.10416666666666667
c : 43 	 all : 0.2986111111111111
s : 44 	 all : 0.3055555555555556
Letter combinations:
 : 73 	 all : 0.5069444444444444


In [10]:
# print metrics without "o"

# iterate through each model and print metrics
print(f"{file_name} without 'o'")
for model in model_prompt_combinations:
    letter_counts = {}
    letter_combinations = {}
    o_instances = 0
    dataset_issues = 0

    for text in labeled_df[model][:sample_size]:
        if "!" in text:
            dataset_issues += 1
            continue 
        
        if "o" in text:
            o_instances += 1
            continue

        if text in letter_combinations:
            letter_combinations[text] += 1
        else:
            letter_combinations[text] = 1

        for letter in text:
            if letter in letter_counts:
                letter_counts[letter] += 1
            else:
                letter_counts[letter] = 1
    print(f"issue type distribution of {model} excluding 'o' instances")
    

    # exclude dataset issues from evaluation
    valid_sample_size = sample_size - dataset_issues - o_instances

    for key in letter_counts:
        percentage = letter_counts[key]/valid_sample_size if valid_sample_size > 0 else 0
        
        print(key, ":", letter_counts[key], "\t all/o :", percentage)
    
    print("Letter combinations:")
    for letter_combination in letter_combinations:
        print(letter_combination, ":", letter_combinations[letter_combination], "\t all/o :", letter_combinations[letter_combination]/valid_sample_size if valid_sample_size > 0 else 0)
    print("..."*10)

spice_label_data_v02.xlsx without 'o'
issue type distribution of llama-7b_zero-shot excluding 'o' instances
Letter combinations:
..............................
issue type distribution of llama-7b_few-shot excluding 'o' instances
i : 89 	 all/o : 0.689922480620155
s : 20 	 all/o : 0.15503875968992248
e : 7 	 all/o : 0.05426356589147287
Letter combinations:
 : 20 	 all/o : 0.15503875968992248
i : 89 	 all/o : 0.689922480620155
se : 7 	 all/o : 0.05426356589147287
s : 13 	 all/o : 0.10077519379844961
..............................
issue type distribution of lora-7b_zero-shot excluding 'o' instances
d : 1 	 all/o : 0.007751937984496124
i : 10 	 all/o : 0.07751937984496124
e : 2 	 all/o : 0.015503875968992248
c : 43 	 all/o : 0.3333333333333333
s : 44 	 all/o : 0.34108527131782945
Letter combinations:
 : 73 	 all/o : 0.5658914728682171
d : 1 	 all/o : 0.007751937984496124
ie : 2 	 all/o : 0.015503875968992248
cs : 42 	 all/o : 0.32558139534883723
s : 2 	 all/o : 0.015503875968992248
i : 8 	

In [11]:
# print metrics without "o" and "s"

# iterate through each model and print metrics
print(f"{file_name} without 'o' and 's'")
for model in model_prompt_combinations:
    letter_counts = {}
    letter_combinations = {}
    s_instances = 0
    o_instances = 0
    dataset_issues = 0

    for text in labeled_df[model][:sample_size]:
        if "!" in text:
            dataset_issues += 1
            continue 

        if "o" in text:
            o_instances += 1
            continue
        
        if "s" in text:
            s_instances += 1
            continue

        if text in letter_combinations:
            letter_combinations[text] += 1
        else:
            letter_combinations[text] = 1

        for letter in text:
            if letter in letter_counts:
                letter_counts[letter] += 1
            else:
                letter_counts[letter] = 1
    print(f"issue type distribution of {model} excluding 'o' and 's' instances")

    # exclude dataset issues from evaluation
    valid_sample_size = sample_size - dataset_issues - s_instances - o_instances

    for key in letter_counts:
        percentage = letter_counts[key]/valid_sample_size
        
        print(key, ":", letter_counts[key], "\t all/s&o :", percentage)
    
    print("Letter combinations:")
    for letter_combination in letter_combinations:
        print(letter_combination, ":", letter_combinations[letter_combination], "\t all/s&o :", letter_combinations[letter_combination]/valid_sample_size)

spice_label_data_v02.xlsx without 'o' and 's'
issue type distribution of llama-7b_zero-shot excluding 'o' and 's' instances
Letter combinations:
issue type distribution of llama-7b_few-shot excluding 'o' and 's' instances
i : 89 	 all/s&o : 0.8165137614678899
Letter combinations:
 : 20 	 all/s&o : 0.1834862385321101
i : 89 	 all/s&o : 0.8165137614678899
issue type distribution of lora-7b_zero-shot excluding 'o' and 's' instances
d : 1 	 all/s&o : 0.011764705882352941
i : 10 	 all/s&o : 0.11764705882352941
e : 2 	 all/s&o : 0.023529411764705882
c : 1 	 all/s&o : 0.011764705882352941
Letter combinations:
 : 73 	 all/s&o : 0.8588235294117647
d : 1 	 all/s&o : 0.011764705882352941
ie : 2 	 all/s&o : 0.023529411764705882
i : 8 	 all/s&o : 0.09411764705882353
c : 1 	 all/s&o : 0.011764705882352941
issue type distribution of lora-7b_few-shot excluding 'o' and 's' instances
d : 1 	 all/s&o : 0.011904761904761904
i : 17 	 all/s&o : 0.20238095238095238
e : 2 	 all/s&o : 0.023809523809523808
Lett